This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
from collections import defaultdict

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
payload = {'api_key': API_KEY, 'limit': '1'}
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json", params=payload)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Get the AFX_X data for the year 2017 by adding the parameters 'start_date' and 'end_date'
payload = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json", params=payload)

In [6]:
#Use .json() method to convert response to dictionary
dict = r.json()

In [7]:
type(dict)

dict

In [8]:
#Extract the column names from the dictionary and store them in the variable keys
keys = dict['dataset_data']['column_names']

In [9]:
#Extract the data from the dictionary and store them in the variable values (list of lists)
values = dict['dataset_data']['data']

In [10]:
#Initialize a new dictionary with default list
data_dict = defaultdict(list)

In [11]:
#Iterate over the range of the data, so that each data point maps to the corresponding key in the 
#dictionary
for i in range(len(values)):
    for j in range(len(keys)):
        data_dict[keys[j]].append(values[i][j])

In [12]:
#Check if any of the values in list for opening prices are None types 
any(isinstance(i, type(None)) for i in data_dict['Open'])

True

In [13]:
#Max and min of the list ascociated with the 'Open' key in our dictionary after removing None types
#Gives us the highest and lowest opening prices for the year
(max([i for i in data_dict['Open'] if i]), min([i for i in data_dict['Open'] if i]))

(53.11, 34.0)

In [14]:
#Check if there are any None types in the High and Low prices
any(isinstance(i, type(None)) for i in data_dict['High'])

False

In [15]:
any(isinstance(i, type(None)) for i in data_dict['Low'])

False

In [16]:
#Make sure both lists are the same length
len(data_dict['High']) == len(data_dict['Low'])

True

In [17]:
#Subtract the prices of the 'Low' list from the 'High' list, then take max of the new list to find 
#largest change
diff = [a - b for a, b in zip(data_dict['High'], data_dict['Low'])]
max(diff)

2.8100000000000023

In [18]:
#Check if all values in 'Traded Volume' are floats
all(isinstance(i, float) for i in data_dict['Traded Volume'])

True

In [19]:
#Sum all the values in 'Traded volume' then divide by the number of values in 'Traded Volume' to find mean
sum(data_dict['Traded Volume'])/len(data_dict['Traded Volume'])

89124.33725490196

In [20]:
len(data_dict['Traded Volume'])

255

In [21]:
#Sort 'Trading Volume' in ascending order
data_dict['Traded Volume'].sort()
tv_sorted = data_dict['Traded Volume']

In [22]:
# We want the middle value because len of "Trading Value" is odd. So add 1 and divide by 2
mid = int((len(data_dict['Traded Volume']) + 1) / 2)
tv_sorted[mid]

76600.0